In [1]:
#Dependencies
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import requests
import os
import pymysql
pymysql.install_as_MySQLdb()
from config import mysqlpassword 

In [2]:
csv_file = "Resources/player_data.csv"
player_data_df = pd.read_csv(csv_file)
player_data_df.head()

,name,year_start,year_end,position,height,weight,birth_date,college
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles"
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University


In [3]:
# Create foreign key to be used for joins based on names (index remains primary key)
player_data_df['namekey'] = player_data_df['name'].str.lower()
for s in [" ", "'", "-", ".", ",", "*"]:
    player_data_df['namekey'] = player_data_df['namekey'].str.replace(s,'')
player_data_df.head()

,name,year_start,year_end,position,height,weight,birth_date,college,namekey
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University,alaaabdelnaby
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University,zaidabdulaziz
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles",kareemabduljabbar
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University,mahmoudabdulrauf
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University,tariqabdulwahad


In [4]:
player_data_df = player_data_df[['name', 'year_start', 'year_end', 'position','birth_date', 'namekey']]
player_data_df.index.name = 'id'
player_data_df.head()

,name,year_start,year_end,position,birth_date,namekey
id,,,,,,
0,Alaa Abdelnaby,1991,1995,F-C,"June 24, 1968",alaaabdelnaby
1,Zaid Abdul-Aziz,1969,1978,C-F,"April 7, 1946",zaidabdulaziz
2,Kareem Abdul-Jabbar,1970,1989,C,"April 16, 1947",kareemabduljabbar
3,Mahmoud Abdul-Rauf,1991,2001,G,"March 9, 1969",mahmoudabdulrauf
4,Tariq Abdul-Wahad,1998,2003,F,"November 3, 1974",tariqabdulwahad


In [5]:
#player_data_df.rename(columns = {})

In [6]:

# dt = datetime.datetime.strptime('%m-%d-%y')

# birth_date = 'birth_date'.format(dt.year, dt.month, dt.day % 100)

In [7]:
# Convert date string to date datatype
player_data_df['birth_date'] = player_data_df['birth_date'].astype('datetime64')
player_data_df.head()

,name,year_start,year_end,position,birth_date,namekey
id,,,,,,
0,Alaa Abdelnaby,1991,1995,F-C,1968-06-24,alaaabdelnaby
1,Zaid Abdul-Aziz,1969,1978,C-F,1946-04-07,zaidabdulaziz
2,Kareem Abdul-Jabbar,1970,1989,C,1947-04-16,kareemabduljabbar
3,Mahmoud Abdul-Rauf,1991,2001,G,1969-03-09,mahmoudabdulrauf
4,Tariq Abdul-Wahad,1998,2003,F,1974-11-03,tariqabdulwahad


In [8]:
# Create Database Connection
rds_connection_string = f"root:{mysqlpassword}@localhost/basketball_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# Confirm tables
engine.table_names()

['player_data', 'players', 'salary']

In [11]:
# Load Dataframe into database
player_data_df.to_sql(name='player_data', con=engine, if_exists='append')

In [12]:
# Confirm data added succesfully
pd.read_sql_query('select * from player_data', con=engine, index_col='id').head()

,name,year_start,year_end,position,birth_date,namekey
id,,,,,,
0,Alaa Abdelnaby,1991,1995,F-C,1968-06-24,alaaabdelnaby
1,Zaid Abdul-Aziz,1969,1978,C-F,1946-04-07,zaidabdulaziz
2,Kareem Abdul-Jabbar,1970,1989,C,1947-04-16,kareemabduljabbar
3,Mahmoud Abdul-Rauf,1991,2001,G,1969-03-09,mahmoudabdulrauf
4,Tariq Abdul-Wahad,1998,2003,F,1974-11-03,tariqabdulwahad
